# **Dropout**

In [2]:
!pip install torch torchvision

## Settings

### 1) Import required libraries


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [4]:
batch_size=256
learning_rate=0.0002
num_epoch=10

## 2. Data

### 1) Download Data

In [5]:
mnist_train=dset.MNIST("C:\\Users\\oh\\Desktop\\민지 대학\\휴먼프밍",train=True,transform=transforms.ToTensor(),target_transform=None,download=True)
mnist_test=dset.MNIST("C:\\Users\\oh\\Desktop\\민지 대학\\휴먼프밍",train=False,transform=transforms.ToTensor(),target_transform=None,download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw/train-images-idx3-ubyte.gz to C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw/train-labels-idx1-ubyte.gz to C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw/t10k-images-idx3-ubyte.gz to C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw/t10k-labels-idx1-ubyte.gz to C:\Users\oh\Desktop\민지 대학\휴먼프밍/MNIST/raw



### 2) Check Dataset

In [14]:
print(mnist_train.__getitem__(0)[0].size(),mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(),mnist_test.__len__())


torch.Size([1, 28, 28]) 60000
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0

### 3) Set DataLoader

In [7]:
train_loader=torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
test_loader=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)

## 3. Model & Optimizer

### 1) CNN Model

In [8]:
from torch.nn.modules.conv import Conv2d
# dropout을 중간중간에 넣어줌 -> 오버피팅 어느정도 극복 가능
# 오버피팅하지 않는 상태에서 드롭아웃 넣으면 오히려 학습이 잘 안 됨.

class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer=nn.Sequential(
        nn.Conv2d(1,16,3,padding=1), # 28
        nn.ReLU(),
        nn.Dropout2d(0.2),
        nn.Conv2d(16,32,3,padding=1), # 28
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.MaxPool2d(2,2), # 14
        nn.Conv2d(32,64,3,padding=1), # 14
        nn.ReLU(),
        nn.Dropout2d(0.2),
        nn.MaxPool2d(2,2) # 7
    ) 
    self.fc_layer=nn.Sequential(
        nn.Linear(64*7*7,100),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(100,10)
    )

  def forward(self,x):
    out=self.layer(x)
    out=out.view(batch_size,-1)
    out=self.fc_layer(out)
    return out

### 2) Loss func & Optimizer

In [9]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model=CNN().to(device)
loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

cuda:0


## 4. Train

In [10]:
for i in range(num_epoch):
  for j, [image,label] in enumerate(train_loader):
    x=image.to(device)
    y_=label.to(device)

    optimizer.zero_grad()
    output=model.forward(x)
    loss=loss_func(output,y_)
    loss.backward()
    optimizer.step()

    if i%10==0:
      print(loss)

tensor(2.3030, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3037, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3127, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3066, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3038, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3022, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3069, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3064, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3024, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3072, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3044, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3038, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3018, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3059, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3008, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3045, device='cuda:0', grad_fn=

In [11]:
param_list=list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[ 0.3068,  0.1680, -0.2659],
          [-0.2375,  0.1035, -0.2909],
          [-0.1322, -0.2331,  0.2366]]],


        [[[ 0.0467, -0.0876,  0.2171],
          [ 0.1365, -0.0777,  0.0192],
          [ 0.2493, -0.3309, -0.2791]]],


        [[[ 0.0936, -0.2487, -0.0803],
          [ 0.1342,  0.1157, -0.2266],
          [ 0.0575,  0.2144,  0.1626]]],


        [[[ 0.0839, -0.2437,  0.2855],
          [-0.0200, -0.3216,  0.0359],
          [-0.2957,  0.2828,  0.1806]]],


        [[[ 0.1374,  0.2901, -0.1864],
          [ 0.1084,  0.1109, -0.1658],
          [-0.1500, -0.2797,  0.1501]]],


        [[[ 0.2163, -0.1618, -0.2993],
          [-0.2882, -0.2480, -0.0934],
          [ 0.0808, -0.2130,  0.2980]]],


        [[[-0.3079,  0.0919, -0.0172],
          [ 0.2295, -0.1907,  0.1008],
          [-0.0685,  0.2816, -0.0231]]],


        [[[ 0.0188, -0.3283,  0.2091],
          [ 0.1042,  0.0815,  0.2277],
          [-0.1365, -0.3148,  0.1782]]],


        [

## 5. Test

In [13]:
correct=0
total=0

# 배치 정규화나 드롭 아웃은 학습할 때와 테스트할 때 다르게 동작 -> 모델을 evaluation 모드로 바꿔서 테스트해야함.
# model.eval(): 학습할 때만 필요한 dropout, batchnorm 등의 기능 비활성화
# torch.no_grad(): gradient 트래킹 X
# https://yuevelyne.tistory.com/10

model.eval()
with torch.no_grad():
  for image,label in test_loader:
    x=image.to(device)
    y_=label.to(device)

    output=model.forward(x)
    _,output_index=torch.max(output,1)

    total+=label.size(0)
    correct+=(output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100*correct/total))


Accuracy of Test Data: 9.57532024383545
